In [1]:
import simtk.openmm.app as app
import simtk.openmm as mm
import simtk.unit as unit
from sys import stdout
from pdbfixer import PDBFixer

**Cargar el archivo PDB del disco y agregar residuos y átomos restantes**:

In [2]:
pdb = PDBFixer("BAK_model1.pdb")

**Agregar residuos, átomos e hidrógenos restantes**:

In [3]:
pdb.findMissingResidues()

pdb.findMissingAtoms()
pdb.addMissingAtoms()

pdb.addMissingHydrogens(7.0)

app.PDBFile.writeFile(pdb.topology, pdb.positions, open("BAK_model1_all.pdb", "w"))

**Especificar el campo de fuerza a utilizar para la simulación**:

In [4]:
forcefield =app.ForceField("amber14-all.xml", "amber14/tip3pfb.xml")

**Crear una descripción matematica completa del sistema por simular**:

In [5]:
system = forcefield.createSystem(pdb.topology, nonbondedMethod=app.NoCutoff, constraints=app.HBonds)

**Crear el integrador**:

Esta línea crea el integrador empleado para avanzar las ecuaciones de movimiento. 

LangevinIntegrator: realiza la dinámica de Langevin, y lo asigna a una variable llamada integrador.

Se declaran los valores de tres parámetros que son específicos de la dinámica de Langevin: la temperatura de simulación (300 K), el coeficiente de fricción (1 ps -1 ) y el tamaño del paso (2 fs).

In [6]:
integrator = mm.LangevinIntegrator(300*unit.kelvin, 1.0/unit.picoseconds, 2.0*unit.femtoseconds)
integrator.setConstraintTolerance(0.00001)

**Preparar la simulación**:

- Definir la plataforma (CUDA o CPU):

In [7]:
platform = mm.Platform.getPlatformByName('CUDA')
properties = {'CudaPrecision': 'mixed'}

- Crear el objeto simulación:

Esta línea combina la topología molecular, el sistema y el integrador para comenzar una nueva simulación.

In [8]:
simulation = app.Simulation(pdb.topology, system, integrator, platform, properties)

In [9]:
simulation.context.setPositions(pdb.positions)

(Esta línea especifica las posiciones iniciales de los átomos para la simulación, en este caso: las posiciones que se cargaron desde el archivo PDB.)

**Minimización de energía**:

In [10]:
print('Minimizing...')
simulation.minimizeEnergy()

Minimizing...


**Equilibrar el sistema a 300 K**:

(50 pasos)

In [11]:
simulation.context.setVelocitiesToTemperature(300*unit.kelvin)
print('Equilibrating...')
simulation.step(50)

Equilibrating...


**Agregar reporteros**:

In [12]:
simulation.reporters.append(app.DCDReporter('trajectory.dcd', 1000))
simulation.reporters.append(app.StateDataReporter(stdout, 1000, step=True,
    potentialEnergy=True, temperature=True, progress=True, remainingTime=True,
    speed=True, totalSteps=500000, separator='\t'))

**Producción**: 1 ns

In [13]:
print('Running Production...')
simulation.step(500000)
print('Done!')

Running Production...
#"Progress (%)"	"Step"	"Potential Energy (kJ/mole)"	"Temperature (K)"	"Speed (ns/day)"	"Time Remaining"
0.2%	1000	-4835.752723336085	283.20857458311207	0	--
0.4%	2000	-4963.104917877699	315.8160147774557	921	1:33
0.6%	3000	-5244.207741625363	310.1280513388491	952	1:30
0.8%	4000	-5234.701537072004	301.00050319687375	968	1:28
1.0%	5000	-5266.5318816505915	292.30166552095153	973	1:27
1.2%	6000	-5026.0944835974005	293.13805892113874	974	1:27
1.4%	7000	-5292.538266950945	312.39026005047276	967	1:28
1.6%	8000	-5149.448170193977	293.1517411490477	910	1:33
1.8%	9000	-5141.460036827334	302.4063366303222	865	1:38
2.0%	10000	-5236.687860152512	290.7705976879079	876	1:36
2.2%	11000	-5206.410962575093	290.8305978899789	886	1:35
2.4%	12000	-5224.446192356804	297.67301328884514	855	1:38
2.6%	13000	-5166.676153440107	318.0773597570849	845	1:39
2.8%	14000	-5085.91056649952	286.4745907094274	852	1:38
3.0%	15000	-5251.874469838861	313.74266532125716	851	1:38
3.2%	16000	-5097.5325076